In [1]:
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import base64
import papermill as pm
import os 
import sys
import re 
import datetime
import pickle
import nbconvert
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert.writers import FilesWriter
%load_ext autoreload
%autoreload 2 

## Run Notebooks

In [2]:
today_str = str(datetime.datetime.now().date())

In [3]:
os.chdir('/home/malcolm/meh_scraper')

In [4]:
tm_notebook = "notebooks/Meh_Analysis_v1.ipynb"
tm_new_notebook = f"notebooks/run_notebooks/Meh_Analysis_v1_{today_str}.ipynb"

In [5]:
try:
    pm.execute_notebook(tm_notebook, tm_new_notebook)
    tm_error = None
except Exception as e:
    tm_error = str(e)
    print("Error")
    print(tm_error)

## Email

In [6]:
sys.path.append('/home/malcolm/EmailSender1/')
from EmailSender import EmailSender as ES

In [12]:
message_params = {}
message_params['Subject'] = "Error in Meh Analysis: "+today_str
message_params['Body'] = """ 
        There was an error in Meh Analysis Notebook. 

        Stack Trace: 
""" + str(tm_error)
message_params['run_notebook_path'] = tm_new_notebook

In [15]:
if tm_error != None:
    email_sender = ES(**message_params)
    email_sender.execute()
    print("Error found")
else: 
    print("No Errors, no error email sent ")

No Errors, no error email sent 
